# 1.IMPORT LIBRARIES

In [1]:
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.preprocessing import LabelEncoder
import mygene
import re
import numpy as np
from warnings import filterwarnings

In [2]:
filterwarnings('ignore')

# 2.PREPROCESSING DATA

## 2.1.GSE91061 DATASET

**GHI CHÚ:** Tạo và xử lý 3 tập dataset của GSE91061 và được lưu trữ dưới định dạng csv:
+ meta_GSE91061.csv: Chứa thông tin meta data của GSE91061
+ RNA_seq_normalized_GSE91061.csv: Chứa thông tin về kết quả `RNA-seq đã được chuẩn hóa`
+ RNA_seq_GSE91061.csv: Chứa thông tin về kết quả `RNA-seq gốc`

In [3]:
# ĐỌC FILE RNA-SEQ GỐC
df_gse91061=pd.read_csv('melanoma/GSE91061/GSE91061_BMS038109Sample.hg19KnownGene.raw.csv',index_col=0).T
df_gse91061

,1,10,100,1000,10000,100009676,10001,10002,10003,100033413,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
Pt1_Pre_AD101148-6,436,0,565,67,6824,239,1045,1,30,0,...,10297,2571,4492,6989,1724,5138,24,3199,1592,8
Pt1_On_AD174047-6,562,1,248,7,10263,186,1063,0,22,0,...,13445,3634,5001,9290,2116,5966,17,3224,2129,2
Pt10_Pre_E9047565-6,306,0,980,338,21827,192,706,11,136,0,...,21486,6304,6441,1218,1661,11114,16,7022,1806,0
Pt10_On_E9047632-6,228,2,727,648,15171,87,639,10,60,5,...,11573,3740,4180,46,1631,8218,7,3011,1333,0
Pt101_Pre_AD486328-5,279,0,1630,228,2485,74,1189,0,19,0,...,11047,3744,1884,1369,4604,9871,9,3233,2398,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pt93_On_AE070988-5,244,0,117,52,8573,92,946,1,36,3,...,13325,2295,4641,8285,2460,9526,24,3543,1223,2
Pt94_Pre_AD732850-6,278,1,515,153,4953,161,641,6,52,0,...,4956,1270,1645,2445,898,3707,13,2647,793,4
Pt94_On_AE373242-6,6387,561,245,1668,2676,101,386,0,57,0,...,3562,910,1093,3260,894,2415,18,2491,526,3
Pt98_Pre_AD733586-8,278,0,212,190,5735,300,766,7,43,0,...,16972,1782,1403,5696,1091,3880,13,2515,643,2


In [4]:
# ĐỌC FILE RNA-SEQ ĐÃ  CHUẨN HÓA 
df_gse91061_normalized=pd.read_csv('melanoma/GSE91061/GSE91061_BMS038109Sample.hg19KnownGene.rld.csv',index_col=0).T
df_gse91061_normalized

,1,10,100,1000,10000,100009676,10001,10002,10003,100033413,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
Pt1_Pre_AD101148-6,9.204906,4.745744,9.006540,6.532238,12.521526,7.585676,9.817017,1.217796,5.074844,-0.122913,...,13.168254,11.165370,11.878346,12.280943,10.547829,12.194201,7.377428,11.476882,10.407948,2.010123
Pt1_On_AD174047-6,9.314037,4.761540,8.010359,4.444946,12.950480,7.246674,9.779680,1.046483,4.720506,-0.123282,...,13.426331,11.516984,11.947953,12.570570,10.732709,12.312469,7.354485,11.428318,10.694963,1.423073
Pt10_Pre_E9047565-6,8.934385,4.744412,9.095214,7.787388,13.347318,6.808879,8.848335,1.827153,6.168423,-0.125874,...,13.477744,11.666228,11.742430,9.639715,9.947586,12.545079,7.335982,11.846273,9.998034,1.095129
Pt10_On_E9047632-6,8.975943,4.779278,9.301396,9.094769,13.481961,6.464609,9.247704,2.034982,5.796133,0.501434,...,13.306694,11.609781,11.792845,6.712429,10.482501,12.750935,7.327581,11.405758,10.198695,1.103528
Pt101_Pre_AD486328-5,9.023143,4.745642,10.226275,7.846224,11.308811,6.260483,9.933494,1.046816,4.599709,-0.123137,...,13.216594,11.575805,10.823513,10.289919,11.691309,12.934810,7.332657,11.454690,10.860502,2.236711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pt93_On_AE070988-5,8.963987,4.745357,7.121161,6.146698,12.654630,6.399411,9.564598,1.199749,5.140935,0.268115,...,13.335693,10.895641,11.778699,12.346926,10.831884,12.788176,7.369626,11.460147,9.961196,1.405319
Pt94_Pre_AD732850-6,9.109138,4.767667,9.163653,7.685573,12.411115,7.395316,9.511627,1.873795,5.886460,-0.120968,...,12.579667,10.607307,10.963672,11.281981,10.048860,12.077931,7.355075,11.521682,9.856848,1.770849
Pt94_On_AE373242-6,19.236138,18.782019,8.506825,10.722709,11.892606,7.068191,9.134363,1.055827,6.175610,-0.119182,...,12.402053,10.424552,10.693571,11.840695,10.247875,11.781885,7.386278,11.656451,9.583739,1.719796
Pt98_Pre_AD733586-8,9.062579,4.746107,7.999846,7.784339,12.431851,7.959032,9.568750,1.888037,5.546967,-0.122121,...,13.881493,10.850841,10.631606,12.150779,10.125388,11.980258,7.349178,11.309542,9.465356,1.464552


In [5]:
# ĐỌC FILE METADATA
with open('melanoma/GSE91061/GSE91061_series_matrix.txt',mode='r',encoding='utf-8') as f:
    lines=f.readlines()

In [6]:
# XỬ LÝ METATA DATA
sample=[]
conditions=[]
for line in lines:
    # LẤY DATA SAMPLE TITLE
    if line.startswith('!Sample_title'):
        sample=re.findall(r'"([^"]+)"',line)
    # LẤY DATA RESPONSSE
    if line.startswith('!Sample_characteristics_ch1') and 'response' in line:
        conditions=re.findall(r'"response:\s*([^"]+)"',line)

In [7]:
# TẠO DATAFRAME META DATA
meta_gse91061=pd.DataFrame({'conditions':conditions},index=sample)
meta_gse91061

,conditions
Pt1_Pre_AD101148-6,PD
Pt1_On_AD174047-6,PD
Pt2_Pre_AD101150-6,SD
Pt2_On_AD174046-6,SD
Pt23_On_AD486568-8,UNK
...,...
Pt106_Pre_AD502250-5,PD
Pt106_On_AD513626-6,PD
Pt34_On_AD485757-6,PRCR
Pt34_Pre_AD466985-6,PRCR


In [52]:
# XÁC ĐỊNH CÁC MẪU 'UNK'
# list_unk_sample_id=meta_gse91061[meta_gse91061['conditions']=='UNK'].index.to_list()
# list_unk_sample_id

['Pt23_On_AD486568-8',
 'Pt23_Pre_AD313075-5',
 'Pt76_Pre_AD667852-6',
 'Pt16_On_AD167378-6']

In [53]:
# LOẠI BỎ CÁC MẪU 'UNK'
# df_gse91061=df_gse91061[~df_gse91061.index.isin(list_unk_sample_id)]
# meta_gse91061=meta_gse91061[~meta_gse91061.index.isin(list_unk_sample_id)]
# df_gse91061_normalized=df_gse91061_normalized[~df_gse91061_normalized.index.isin(list_unk_sample_id)]

In [13]:
# TẠO HÀM ĐỂ GẮN NHÃN MỚI CHO 'METADATA'
def Responses_categorazation(name: str)->str:
    if name=='PRCR' or name=='UNK':
        return 'Non-responders'
    return 'Responder'

In [14]:
# GÁN LẠI NHÃN CHO 'METADATA'
meta_gse91061['New_conditions']=meta_gse91061['conditions'].apply(Responses_categorazation)
meta_gse91061

,conditions,New_conditions
Pt1_Pre_AD101148-6,PD,Responder
Pt1_On_AD174047-6,PD,Responder
Pt2_Pre_AD101150-6,SD,Responder
Pt2_On_AD174046-6,SD,Responder
Pt23_On_AD486568-8,UNK,Non-responders
...,...,...
Pt106_Pre_AD502250-5,PD,Responder
Pt106_On_AD513626-6,PD,Responder
Pt34_On_AD485757-6,PRCR,Non-responders
Pt34_Pre_AD466985-6,PRCR,Non-responders


In [15]:
# ENCODING 
meta_gse91061['New_conditions_encoded']=(meta_gse91061['New_conditions']=='Responder').astype('int')
meta_gse91061

,conditions,New_conditions,New_conditions_encoded
Pt1_Pre_AD101148-6,PD,Responder,1
Pt1_On_AD174047-6,PD,Responder,1
Pt2_Pre_AD101150-6,SD,Responder,1
Pt2_On_AD174046-6,SD,Responder,1
Pt23_On_AD486568-8,UNK,Non-responders,0
...,...,...,...
Pt106_Pre_AD502250-5,PD,Responder,1
Pt106_On_AD513626-6,PD,Responder,1
Pt34_On_AD485757-6,PRCR,Non-responders,0
Pt34_Pre_AD466985-6,PRCR,Non-responders,0


In [16]:
# SORT THỨ TỰ CÁC CỘT THEO INDEX
df_gse91061.sort_index(ascending=True,inplace=True)
df_gse91061_normalized.sort_index(ascending=True,inplace=True)
meta_gse91061.sort_index(ascending=True,inplace=True)

In [17]:
# Lưu file tại thư mục Preproccessed_data
df_gse91061_normalized.to_csv(path_or_buf='Preprocessed_data/GSE91061/RNA_seq_normalized_GSE91061.csv')
df_gse91061.to_csv(path_or_buf='Preprocessed_data/GSE91061/RNA_seq_GSE91061.csv')
meta_gse91061.to_csv(path_or_buf='Preprocessed_data/GSE91061/meta_GSE91061.csv')

## 2.2.GSE78220 DATASET

**GHI CHÚ:** Tạo và xử lý 2 tập dataset của GSE78220 và được lưu trữ dưới định dạng csv:
+ meta_GSE78220.csv: Chứa thông tin meta data của GSE78220
+ RNA_seq_normalized_GSE78220.csv: Chứa thông tin về kết quả `RNA-seq đã được chuẩn hóa`

In [59]:
# ĐỌC FILE RNA-SEQ ĐÃ CHUẨN HÓA
df_gse78220=pd.read_excel('melanoma/GSE78220/GSE78220_PatientFPKM.xlsx',index_col=0).T
df_gse78220

Gene,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
Pt1.baseline,8.025380,0.677503,0.000000,185.30400,0.363531,0.014134,0.068722,0.000000,6.557490,0.024924,...,4.517615,9.838935,1.694975,3.152445,7.699230,0.007498,7.302015,61.04475,3.463620,14.703500
Pt2.baseline,317.594000,28.378200,18.109400,455.16000,2.864455,0.012692,0.024855,0.036931,0.472773,0.107477,...,4.114925,7.464380,2.130045,4.779425,7.368425,0.054845,5.610695,37.38650,3.190875,9.412335
Pt4.baseline,5.747295,0.471394,0.000000,81.00820,0.201285,0.022505,0.033771,0.019492,0.610117,0.010961,...,12.002800,21.035850,1.578830,4.141195,11.295950,0.000000,4.003770,54.04650,4.797630,6.409365
Pt5.baseline,6.859570,0.347623,0.000000,41.41065,0.040821,0.003837,0.000000,0.000000,0.624096,0.000000,...,10.062785,14.672800,1.481260,3.334790,8.061355,0.000000,5.655590,48.22575,4.612085,9.785310
Pt6.baseline,1.563465,0.201760,0.011276,136.16350,0.417423,0.000000,0.013876,0.000000,0.991819,0.000000,...,14.114750,13.622750,2.993685,5.950100,6.672260,0.000000,5.824330,69.13510,6.704615,9.316395
Pt7.baseline,21.565850,8.926245,0.000000,44.66935,1.039475,0.010840,0.039277,0.000000,1.531155,0.012505,...,10.309600,24.090500,1.051455,2.619185,12.015400,2.066555,4.543575,26.51905,3.711905,6.672100
Pt8.baseline,11.370550,3.853125,0.003263,119.01750,3.126025,0.044663,0.065869,0.000000,6.753445,0.000000,...,6.325255,10.055250,0.849893,1.763070,7.663970,0.000000,3.886960,177.41800,5.336710,8.664665
Pt9.baseline,8.156810,2.539610,0.001398,232.67500,4.216800,0.447586,0.276618,0.077716,4.764565,0.051647,...,7.254080,15.598700,1.264025,2.488195,6.597545,0.006864,4.239625,72.37300,4.941620,6.003945
Pt10.baseline,4.346750,1.914350,0.000000,501.69700,1.489845,0.030801,0.056148,0.178040,26.199650,0.025032,...,7.060635,18.511250,1.000603,1.562540,8.046115,0.022734,5.457640,71.65415,3.896335,7.293775
Pt12.baseline,6.508715,4.797930,0.000000,530.23750,1.868520,0.007370,0.018405,0.059667,2.187715,0.248870,...,24.105250,38.137750,0.407459,1.856880,4.742560,0.010006,3.121325,379.13500,3.783140,11.620600


In [60]:
# ĐỌC FILE META DATA
with open('melanoma/GSE78220/GSE78220_series_matrix.txt',mode='r',encoding='utf-8') as f:
    lines=f.readlines()

In [61]:
# XỬ LÝ FIEL META DATA
sample=[]
conditions=[]
for line in lines:
    # LẤY DATA SAMPLE TITLE
    if line.startswith('!Sample_title'):
        sample=re.findall(r'"([^"]+)"',line)
    # LẤY DATA TREATMENT RESPONSE
    if line.startswith('!Sample_characteristics_ch1') and 'anti-pd-1 response' in line:
        conditions=re.findall(r'"anti-pd-1 response:\s*([^"]+)"',line)

In [62]:
# TẠO DATAFRAME METADATA
meta_gse78220=pd.DataFrame({'conditions':conditions},index=sample)
meta_gse78220

,conditions
Pt1,Progressive Disease
Pt2,Partial Response
Pt4,Partial Response
Pt5,Partial Response
Pt6,Partial Response
Pt7,Progressive Disease
Pt8,Complete Response
Pt9,Complete Response
Pt10,Progressive Disease
Pt12,Progressive Disease


In [63]:
# TẠO HÀM ĐỂ GẮN NHÃN CHO 'METADATA'
def Responses_categorazation(name: str)->str:
    if name=='Progressive Disease':
        return 'Non-responders'
    return 'Responder'

In [64]:
# GÁN LẠI NHÃN CHO 'METADATA'
meta_gse78220['New_conditions']=meta_gse78220['conditions'].apply(Responses_categorazation)
meta_gse78220

,conditions,New_conditions
Pt1,Progressive Disease,Non-responders
Pt2,Partial Response,Responder
Pt4,Partial Response,Responder
Pt5,Partial Response,Responder
Pt6,Partial Response,Responder
Pt7,Progressive Disease,Non-responders
Pt8,Complete Response,Responder
Pt9,Complete Response,Responder
Pt10,Progressive Disease,Non-responders
Pt12,Progressive Disease,Non-responders


In [65]:
# ENCODING 
meta_gse78220['New_conditions_encoded']=(meta_gse78220['New_conditions']=='Responder').astype('int')
meta_gse78220

,conditions,New_conditions,New_conditions_encoded
Pt1,Progressive Disease,Non-responders,0
Pt2,Partial Response,Responder,1
Pt4,Partial Response,Responder,1
Pt5,Partial Response,Responder,1
Pt6,Partial Response,Responder,1
Pt7,Progressive Disease,Non-responders,0
Pt8,Complete Response,Responder,1
Pt9,Complete Response,Responder,1
Pt10,Progressive Disease,Non-responders,0
Pt12,Progressive Disease,Non-responders,0


In [66]:
# SORT THỨ TỰ CÁC CỘT THEO INDEX
df_gse78220.sort_index(ascending=True,inplace=True)
meta_gse78220.sort_index(ascending=True,inplace=True)

In [67]:
# Lưu hai file
df_gse78220.to_csv(path_or_buf='Preprocessed_data/GSE78220/RNA_seq_normalized_GSE78220.csv')
meta_gse78220.to_csv(path_or_buf='Preprocessed_data/GSE78220/meta_GSE78220.csv')